In [1]:
import cv2
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.segmentation import flood, flood_fill


In [2]:
def Gamma_Correction(img , c , gamma):
    img2 = np.copy(img)
    img2 = c*np.power(img , gamma)
    return img2

def erosion(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])==n*n        
    return img2

def dilation(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])>=1        
    return img2

In [3]:
def multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame):
    
    # Extracting the blue, red, green and alpha channels
    
    B = rgba_frame[:,:,0]
    G = rgba_frame[:,:,1]
    R = rgba_frame[:,:,2]
    A = rgba_frame[:,:,3]
    
    #Extracting the Hue, Saturation and vue channels
    
    H = hsv_frame[:,:,0]
    S = hsv_frame[:,:,1]
    V = hsv_frame[:,:,2]
    
    # Extracting the Y, Cr and Cb channels
    
    Y = ycbcr_frame[:,:,0]
    Cr = ycbcr_frame[:,:,1]
    Cb = ycbcr_frame[:,:,2]


    # Applying  Thresholding using Log-Chromaticity color space 
#     n_r = np.ma.divide(R, G)
#     n_b = np.ma.divide(B, G)
#     log_rg = np.ma.log( n_r )
#     log_bg = np.ma.log( n_b )
#     condition_1 = (log_rg>=0.15) & (log_rg<=1.1)
#     condition_2 = (log_bg>=-4) & (log_bg<=0.3)
#     mask_1 = condition_1 & condition_2
    
    # Extracting masks based on a combination of RGBA, HSV and YCrCb models for skin detection
    
    mask_rgb = (R>95)&(G>40)&(B>20)&(R>G)&(R>B)&(abs(R-G)>15)&(A>15)
    mask_Ycbcr = (Cr > 135)&(Cb>85)&(Y>80)&(Cr <= (1.5862*Cb)+20)&(Cr>=(0.3448*Cb)+76.2069)&(Cr >= (-4.5652*Cb)+234.5652)&(
                  Cr <= (-1.15*Cb)+301.75)&(Cr <= (-2.2857*Cb)+432.85)
    mask_hsv = ((0.0 <= H) & (H <= 50.0))&((0.23 <= S) &(S <= 0.68))
    
    mask1 = mask_rgb & mask_Ycbcr
    mask2 = mask_rgb & mask_hsv
    
    return mask1 | mask2
    

In [4]:
def find_centers_of_contours(binary_image):
    contours = find_contours(binary_image , 0.8)
    centers = []
    bounding_boxes = list()
    for i in range(len(contours)):
        Xmin = int(min(contours[i][:,1]))
        Xmax = int(max(contours[i][:,1]))
        Ymin = int(min(contours[i][:,0]))
        Ymax = int(max(contours[i][:,0]))
        if( 100<=(Xmax-Xmin)*(Ymax-Ymin)and (Xmax-Xmin)*(Ymax-Ymin) <= 500):
            bounding_boxes.append([Xmin , Xmax , Ymin , Ymax])
#             moments = cv2.moments(contours[i])
#             centers.append((int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00'])))
#             cv2.circle(binary_image, centers[-1], 3, (0, 0, 0), -1)
    return bounding_boxes

In [5]:
def draw_contours(bounding_boxes , img):
    
    for box in bounding_boxes:
        [Xmin, Xmax, Ymin, Ymax] = box
        rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img.shape)
        img[rr, cc] = 1
    pass
        

In [6]:
def moving_area():
    
    frame_number = 0
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    gray_frame = rgb2gray(rgb_frame) 
    acc_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    
    while (frame_number < 0):
        ret, cur_frame = cap.read()
        cur_frame = cv2.resize(cur_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
        cur_frame = rgb2gray(cur_frame)
        acc_frame += cur_frame
        frame_number += 1 
        
    acc_frame /= frame_number
    diff_frame = abs(acc_frame-gray_frame)
    moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    moving_region[diff_frame > 0.2] = 1
    one_positions = np.argwhere(moving_region == 1)
    if(len(one_positions) > 0):
        Xmin = min(one_positions[:,0])-20
        Xmax = max(one_positions[:,0])+20
        Ymin = min(one_positions[:,1])-120
        Ymax = max(one_positions[:,1])+120
        moving_region[Xmin:Xmax , Ymin:Ymax] = 1
    
    return moving_region
    

In [7]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

old_moving_region = [] 

while True:
    
#     face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    
#     gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     for (x,y,w,h) in faces: 
#         # To draw a rectangle in a face  
#         cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)   
     
            
    moving_region = moving_area() 
    if(np.sum(moving_region) < 1000):
        moving_region = old_moving_region
    old_moving_region = moving_region
    
    # converting the rgb space to hsv space color
    hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
    # converting the rgb space to YCbCr space color
    ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
    # converintg the rgb space to rgba spave 
    rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
    
    
    # the final mask to extract the binary image
    mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
    
    binary_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    mask2 = moving_region==1
    binary_frame[mask&mask2] = 1
    binary_frame2[mask] = 1
    
    #removing noise
#     eroded_frame = binary_erosion(binary_frame2 , np.ones((5,5)) )
#     dilated_frame = binary_dilation(eroded_frame , np.ones((10,10)) )
#     dilated_frame = binary_dilation(dilated_frame , np.ones((5,5)) )

    final_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    final_frame[eroded_frame] = 1
    cv2.imshow('Input', rgb_frame)
    cv2.imshow('Output', binary_frame2)
    cv2.imshow('Output1', final_frame)
    
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()


/home/ahmed/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/home/ahmed/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater
